# Create training samples for the map-matching error detection model

In [ ]:
import os
import copy
import pickle
import numpy as np
import pandas as pd

from tqdm import tqdm, trange

import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, LineString

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset

from utils import cal_d_hdg, normalize, create_samples, calc_pos_weight, organize_trajs
from utils import tensorize_input, MatchErrorDataset, train_val_test_set

In [ ]:
proj_dir = "<YOUR_PROJECT_DIRECTORY>"

## Real samples

In [ ]:
# Load pickle
pk_name = "labeled184_pts_traj_ptid.pkl"
with open(
    os.path.join(proj_dir, "Data", pk_name), 'rb'
) as my_file_obj:
    pts_gdf, traj_ptid_ls = pickle.load(my_file_obj)

print("Number of trajectories:", len(traj_ptid_ls))
print("Number of points:", len(pts_gdf))
pts_gdf.head()

### Map-matching error statistics

In [ ]:
n_pts = len(pts_gdf)
n_ones = sum(pts_gdf["mat_error"])
n_zeros = n_pts - n_ones

labeled_stats_df = pd.DataFrame(
    {
        "correct": n_zeros, "wrong": n_ones, "n_pts": n_pts,
        "error_rate": round(n_ones / n_pts, 4)
    }, index=[0]
)

labeled_stats_df

### Road network attributes

In [ ]:
# Load road network graph
graph_name = "aa_road_graph_drive_service_bbox_time_speed_bearing.graphml"  # Ann Arbor
roadnet_graph = ox.load_graphml(
    os.path.join(proj_dir, graph_name)
)
print("Road graph loaded!")

# Graph to gdf
nodes_gdf, edges_gdf = ox.graph_to_gdfs(roadnet_graph)
print("nodes_gdf, edges_gdf created!")

Some roads (edges) have more than one class in the "highway" field, we choose its first class that is not "unclassified".

In [ ]:
edges_gdf["edge_class"] = copy.deepcopy(edges_gdf["highway"])

for edge_i in trange(len(edges_gdf)):
    edge_i_class = edges_gdf.loc[edge_i, "edge_class"]
    if isinstance(edge_i_class, list):
        class_j = "unclassified"  # Initialize class_j
        for class_j in edge_i_class:
            if class_j == "unclassified":
                continue
            else:
                break
        edges_gdf.loc[edge_i, "edge_class"] = class_j

Merge "services" with "service" in the edge class.

In [ ]:
# Rename services to service
edges_gdf.loc[edges_gdf["edge_class"] == "services", "edge_class"] = "service"

In [ ]:
# Overview of all the edges
edges_gdf.groupby(by=["edge_class"]).count()

### Point attributes

Associate each matched tracking point to its closest edge.

In [ ]:
# Spatial index
spatial_idx = edges_gdf.sindex

closest_edges_ls = []
for pt_i in trange(len(pts_gdf)):
    # Create a buffer to find possible matched edges
    buffer_dist = 1000
    pt_i_buffer = pts_gdf.loc[pt_i, "geometry"].buffer(buffer_dist)
    possible_matches_idx_ls = list(spatial_idx.intersection(pt_i_buffer.bounds))

    # Retrieve the possible matched edges from the GeoDataFrame
    possible_matches_gdf = edges_gdf.iloc[possible_matches_idx_ls]

    # Calculate distances from the point to these edges
    distances = possible_matches_gdf.distance(pts_gdf.iloc[pt_i]["geometry"])

    # Find the closest edge
    closest_edge_idx = distances.idxmin()
    closest_edges_ls.append((pt_i, closest_edge_idx, distances[closest_edge_idx]))

# Convert the list of closest edges to a DataFrame for easier handling
closest_edges_df = pd.DataFrame(closest_edges_ls, columns=['pt_idx', 'closest_edge_idx', 'distance'])

In [ ]:
# Difference between two consecutive headings
pts_gdf["d_hdg"] = 0.0  # Delta heading
# Difference between the heading and road bearing
pts_gdf["d_hdg_brg"] = 0.0  # Delta heading-bearing
# Distance to the closest road
pts_gdf["dist2road"] = 1000.0
# Edge class
pts_gdf["edge_class"] = "unclassified"

for traj_i in tqdm(traj_ptid_ls):
    for pt_count in range(len(traj_i[1])):
        pt_j = traj_i[1][pt_count]
        # Delta heading
        if pt_count == 0:  # 1st pt in a trajectory
            pts_gdf.loc[pt_j, "d_hdg"] = 0
        else:
            pt_j_prev1 = traj_i[1][pt_count-1]  # Previous pt
            pts_gdf.loc[pt_j, "d_hdg"] = cal_d_hdg(
                hdg1=pts_gdf.loc[pt_j, "heading"], hdg2=pts_gdf.loc[pt_j_prev1, "heading"]
            )
        # Delta heading-bearing
        closest_edge_idx = closest_edges_df.loc[pt_j, "closest_edge_idx"]
        pts_gdf.loc[pt_j, "d_hdg_brg"] = cal_d_hdg(
            hdg1=pts_gdf.loc[pt_j, "heading"], hdg2=edges_gdf.loc[closest_edge_idx, "bearing"]
        )
        # Distance to the closest road
        pts_gdf.loc[pt_j, "dist2road"] = closest_edges_df.loc[pt_j, "distance"]
        # Edge class
        pts_gdf.loc[pt_j, "edge_class"] = edges_gdf.loc[closest_edge_idx, "edge_class"]

# Fill nan with 90
pts_gdf = pts_gdf.fillna(90)
# Double check nan
nan_row_ids = pts_gdf.isna().any(axis=1)

Convert `pts_gdf["edge_class"]` from categorical strings to encoding values.

In [ ]:
pts_gdf['edge_class_encoded'] = pd.Categorical(
    pts_gdf['edge_class'],
    categories=[
        "living_street", "motorway", "motorway_link", "primary", "primary_link", "residential",
        "secondary", "secondary_link", "service", "tertiary", "tertiary_link", "trunk", "unclassified"
    ]
).codes

### Normalization

In [ ]:
columns2norm_ls = ["d_hdg", "d_hdg_brg", "dist2road"]
target_range_tp_ls = [(-180, 180), (-180, 180), (0, 500)]

for col_count, col_i in enumerate(columns2norm_ls):
    print("Now normalizing column:", col_i)
    x_arr = pts_gdf[col_i].to_numpy()
    x_norm_arr = normalize(x_arr, orig_range_tp=target_range_tp_ls[col_count], target_range_tp=(0.1, 1))

    # Add normalized values back to pts_df
    new_col_name = col_i + "_norm"
    pts_gdf[new_col_name] = x_norm_arr

    print(f"Range after normalization: ({round(min(x_norm_arr), 3)}, {round(max(x_norm_arr), 3)})")

### Samples

In [ ]:
samp_pts_ls = create_samples(
    min_seq_len=10, max_seq_len=100, stride=2, traj_ptid_ls=traj_ptid_ls
)

In [ ]:
x_ls = []
y_ls = []

for sample_i in tqdm(samp_pts_ls):
    # Sample input
    x_i = pts_gdf.loc[sample_i, ["d_hdg", "d_hdg_brg", "dist2road", "edge_class_encoded"]].to_numpy()
    # Sample output
    y_i = pts_gdf.loc[sample_i, "mat_error"].to_numpy()

    x_ls.append(x_i)
    y_ls.append(y_i)

print(f"x_ls len: {len(x_ls)}, y_ls len: {len(y_ls)}")

Calculate the positive weight pos_weight, a parameter needed in the weighted cross-entropy loss.

In [ ]:
weight_for_class_1 = round(calc_pos_weight(y_ls), 4)
weight_for_class_1

#### Training, validation, and test sets

In [ ]:
train_set, val_set, test_set = train_val_test_set(x_ls, y_ls, train_pct=0.7)

print(f"Train set size: {len(train_set)}")
print(f"Val set size: {len(val_set)}")
print(f"Test set size: {len(test_set)}")

In [ ]:
# Save to pickle
pk_name = "train_val_test_set_stride2_labeled184.pkl"
with open(os.path.join(proj_dir, pk_name), 'wb') as my_file_obj:
    pickle.dump([train_set, val_set, test_set], my_file_obj)

## Synthetic samples

In [ ]:
# Time interval
tint = 5

# Load pickle
pk_name = f"track_pts_tint{tint}_ntimes10k_fixeddist20_error02.pkl"
with open(os.path.join(proj_dir, pk_name), 'rb') as my_file_obj:
    pts_gdf = pickle.load(my_file_obj)

traj_ptid_ls = organize_trajs(pts_df=pts_gdf, use_traj_id=True)

print("Number of trajectories:", len(traj_ptid_ls))
print("Number of points:", len(pts_gdf))
pts_gdf.head()

### Road network attributes

In [ ]:
# Load road network graph
graph_name = "aa_road_graph_drive_service_bbox_time_speed_bearing.graphml"  # Ann Arbor
# graph_name = "la_road_graph_drive_service_bbox_time_speed_bearing.graphml"  # Los Angeles
roadnet_graph = ox.load_graphml(
    os.path.join(proj_dir, graph_name)
)
print("Road graph loaded!")

# Graph to gdf
nodes_gdf, edges_gdf = ox.graph_to_gdfs(roadnet_graph)
print("nodes_gdf, edges_gdf created!")

Some roads (edges) have more than one class in the "highway" field, we choose its first class that is not "unclassified".

In [ ]:
edges_gdf["edge_class"] = copy.deepcopy(edges_gdf["highway"])

for edge_i in trange(len(edges_gdf)):
    edge_i_class = edges_gdf.loc[edge_i, "edge_class"]
    if isinstance(edge_i_class, list):
        class_j = "unclassified"  # Initialize class_j
        for class_j in edge_i_class:
            if class_j == "unclassified":
                continue
            else:
                break
        edges_gdf.loc[edge_i, "edge_class"] = class_j

Merge "services" with "service" in the edge class.

In [ ]:
# Rename services to service
edges_gdf.loc[edges_gdf["edge_class"] == "services", "edge_class"] = "service"

In [ ]:
# Overview of all the edges
edges_gdf.groupby(by=["edge_class"]).count()

### Point attributes

In [ ]:
# Spatial index
spatial_idx = edges_gdf.sindex

closest_edges_ls = []
for pt_i in trange(len(pts_gdf)):
    # Create a buffer to find possible matched edges
    buffer_dist = 1000
    pt_i_buffer = pts_gdf.loc[pt_i, "geometry"].buffer(buffer_dist)
    possible_matches_idx_ls = list(spatial_idx.intersection(pt_i_buffer.bounds))

    # Retrieve the possible matched edges from the GeoDataFrame
    possible_matches_gdf = edges_gdf.iloc[possible_matches_idx_ls]

    # Calculate distances from the point to these edges
    distances = possible_matches_gdf.distance(pts_gdf.iloc[pt_i]["geometry"])

    # Find the closest edge
    closest_edge_idx = distances.idxmin()
    closest_edges_ls.append((pt_i, closest_edge_idx, distances[closest_edge_idx]))

# Convert the list of closest edges to a DataFrame for easier handling
closest_edges_df = pd.DataFrame(closest_edges_ls, columns=['pt_idx', 'closest_edge_idx', 'distance'])

In [ ]:
# Difference between two consecutive headings
pts_gdf["d_hdg"] = 0.0  # Delta heading
# Difference between the heading and road bearing
pts_gdf["d_hdg_brg"] = 0.0  # Delta heading-bearing
# Distance to the closest road
pts_gdf["dist2road"] = 1000.0
# Edge class
pts_gdf["edge_class"] = "unclassified"

for traj_i in tqdm(traj_ptid_ls):
    for pt_count in range(len(traj_i[1])):
        pt_j = traj_i[1][pt_count]
        # Delta heading
        if pt_count == 0:  # 1st pt in a trajectory
            pts_gdf.loc[pt_j, "d_hdg"] = 0
        else:
            pt_j_prev1 = traj_i[1][pt_count-1]  # Previous pt
            pts_gdf.loc[pt_j, "d_hdg"] = cal_d_hdg(
                hdg1=pts_gdf.loc[pt_j, "heading"], hdg2=pts_gdf.loc[pt_j_prev1, "heading"]
            )
        # Delta heading-bearing
        closest_edge_idx = closest_edges_df.loc[pt_j, "closest_edge_idx"]
        pts_gdf.loc[pt_j, "d_hdg_brg"] = cal_d_hdg(
            hdg1=pts_gdf.loc[pt_j, "heading"], hdg2=edges_gdf.loc[closest_edge_idx, "bearing"]
        )
        # Distance to the closest road
        pts_gdf.loc[pt_j, "dist2road"] = closest_edges_df.loc[pt_j, "distance"]
        # Edge class
        pts_gdf.loc[pt_j, "edge_class"] = edges_gdf.loc[closest_edge_idx, "edge_class"]

# Fill nan with 90
pts_gdf = pts_gdf.fillna(90)
# Double check nan
nan_row_ids = pts_gdf.isna().any(axis=1)

In [ ]:
pts_gdf['edge_class_encoded'] = pd.Categorical(
    pts_gdf['edge_class'],
    categories=[
        "living_street", "motorway", "motorway_link", "primary", "primary_link", "residential",
        "secondary", "secondary_link", "service", "tertiary", "tertiary_link", "trunk", "unclassified"
    ]
).codes

### Normalization

In [ ]:
columns2norm_ls = ["d_hdg", "d_hdg_brg", "dist2road"]
target_range_tp_ls = [(-180, 180), (-180, 180), (0, 500)]

for col_count, col_i in enumerate(columns2norm_ls):
    print("Now normalizing column:", col_i)
    x_arr = pts_gdf[col_i].to_numpy()
    x_norm_arr = normalize(x_arr, orig_range_tp=target_range_tp_ls[col_count], target_range_tp=(0.1, 1))

    # Add normalized values back to pts_df
    new_col_name = col_i + "_norm"
    pts_gdf[new_col_name] = x_norm_arr

    print(f"Range after normalization: ({round(min(x_norm_arr), 3)}, {round(max(x_norm_arr), 3)})")

# Map original labels 0, 1, 2 to 0, 1
pts_gdf["error01"] = (pts_gdf["mat_error"] > 0).astype("int")

### Samples

In [ ]:
samp_pts_ls = create_samples(
    min_seq_len=10, max_seq_len=100, stride=20, traj_ptid_ls=traj_ptid_ls
)

In [ ]:
x_ls = []
y_ls = []

for sample_i in tqdm(samp_pts_ls):
    # Sample input
    x_i = pts_gdf.loc[sample_i, ["d_hdg", "d_hdg_brg", "dist2road", "edge_class_encoded"]].to_numpy()
    # Sample output
    y_i = pts_gdf.loc[sample_i, "error01"].to_numpy()

    x_ls.append(x_i)
    y_ls.append(y_i)

print(f"x_ls len: {len(x_ls)}, y_ls len: {len(y_ls)}")

Calculate the positive weight pos_weight, a parameter needed in the weighted cross-entropy loss.

In [ ]:
weight_for_class_1 = round(calc_pos_weight(y_ls), 4)
weight_for_class_1

#### Training, validation, and test sets

In [ ]:
train_set, val_set, test_set = train_val_test_set(x_ls, y_ls, train_pct=0.7)

print(f"Train set size: {len(train_set)}")
print(f"Val set size: {len(val_set)}")
print(f"Test set size: {len(test_set)}")

In [ ]:
# Save to pickle
pk_name = f"train_val_test_set_stride20_tint{tint}_ntimes10k.pkl"
with open(
        os.path.join(proj_dir, pk_name), 'wb'
) as my_file_obj:
    pickle.dump([train_set, val_set, test_set], my_file_obj)